In [356]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('../cultura_1.db')
cursor = conn.cursor()

df_object = pd.read_sql_query("SELECT * FROM individual_created_work", conn)
df_instance = pd.read_sql_query("SELECT * FROM created_work", conn)
df_ind_object = pd.merge(df_object, df_instance, on = ['work_wikidata_id', 'work_name'], how = 'left')

In [437]:
df_individual = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_individual = df_individual[['individual_wikidata_id', 'birthyear']]

df_individual_region = pd.read_sql_query("SELECT * FROM individuals_regions", conn)
df_individual_region = df_individual_region[['individual_wikidata_id', 'region_name']].drop_duplicates()

df_meta = pd.merge(df_individual, df_individual_region, on = 'individual_wikidata_id')
df_meta['productive_year'] = df_meta['birthyear'] + 35

In [358]:
df = pd.merge(df_ind_object, df_meta, on = 'individual_wikidata_id')

In [359]:
# 373473 objects with a date and a country

In [360]:
df['year'] = df.apply(lambda row: row['creation_year'] if row['creation_year'] is not None else row['productive_year'], axis=1)
df = df.dropna(subset= 'year')
df['decade'] = df['year'].apply(lambda x : int(x/10)*10)

In [366]:
df_trend = df[['work_wikidata_id', 'region_name', 'decade', 'work_category']].drop_duplicates()
df_trend = df_trend.groupby(['work_category', 'region_name', 'decade'])['work_wikidata_id'].count().rename('score').reset_index()
df_trend = df_trend[df_trend['decade']<=1850].reset_index(drop=True)

In [369]:
df_trend.to_csv('data_to_r/df_trend_objects_categories.csv')

In [334]:
len(set(df.work_wikidata_id))

373473

In [335]:
# if not the year, then get the pic year of the individual (birthyear + 35)

In [372]:
#df_clean_categories = df_instance[df_instance['work_wikidata_id'].isin(list(set(df.work_wikidata_id)))]
#df_clean_categories['super_instance_label'][df_clean_categories['super_instance_label'].isna()] = ''
#df_clean_categories['instance_label'][df_clean_categories['instance_label'].isna()] = ''
#df_clean_categories = df_clean_categories.drop(['work_name', 'inception', 'publication_date'], axis=1).drop_duplicates()


"""
df_clean = df_clean_categories.groupby(['instance_label', 'super_instance_label'])['work_wikidata_id'].count().rename('count_all').reset_index()
df_clean = df_clean.groupby('instance_label').agg({'super_instance_label': lambda x: " | ".join(list(x)), 'count_all': 'sum'}).reset_index()
df_clean = df_clean.sort_values('count_all', ascending=False).reset_index(drop=True)
df_clean.to_clipboard(index=False)
"""

'\ndf_clean = df_clean_categories.groupby([\'instance_label\', \'super_instance_label\'])[\'work_wikidata_id\'].count().rename(\'count_all\').reset_index()\ndf_clean = df_clean.groupby(\'instance_label\').agg({\'super_instance_label\': lambda x: " | ".join(list(x)), \'count_all\': \'sum\'}).reset_index()\ndf_clean = df_clean.sort_values(\'count_all\', ascending=False).reset_index(drop=True)\ndf_clean.to_clipboard(index=False)\n'

In [375]:
import plotly.express as px

In [400]:
set(df_trend.region_name)

{'Arabic world',
 'Austria',
 'Balkans',
 'Belgium',
 'British Islands',
 'Central Europe',
 'Chinese world',
 'Denmark',
 'Eastern Europe',
 'Finland',
 'France',
 'German world',
 'Germany',
 'Greece',
 'Greek World',
 'Iceland',
 'Indian world',
 'Ireland',
 'Italy',
 'Japan',
 'Korea',
 'Latin World',
 'Low countries',
 'Muslim world',
 'Netherlands',
 'Nordic countries',
 'Northern China',
 'Northern France',
 'Northern India',
 'Northern Italy',
 'Northern Japan',
 'Northwestern Europe',
 'Norway',
 'Ottoman Turkey',
 'Ottoman world',
 'Persian world',
 'Portugal',
 'Slav world',
 'South East Asia',
 'Southern China',
 'Southern France',
 'Southern India',
 'Southern Italy',
 'Southern Japan',
 'Southwestern Europe',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'Western Europe',
 'Yangtze',
 'mediterranean World'}

In [383]:
def make_figure(df, region_name = 'Low countries', min_decade = 1200, max_decade = 1800,  work_category = ['musical work/composition']):

    df_fig = df[df['region_name']==region_name]
    df_fig = df_fig[df_fig['decade']<=max_decade]
    df_fig = df_fig[df_fig['decade']>=min_decade]
    #df_fig = df_fig[df_fig['instance_label'].isin(category)]

    #top_ids = df_fig.groupby(['instance_label'])['score'].sum().reset_index().sort_values('score', ascending=False)

    #top_ids = top_ids[top_ids['identifier_name']!='all_identifiers']
    #top_ids = list(top_ids['instance_label'][:10])
    #df_fig = df_fig[df_fig['instance_label'].isin(top_ids)]
    
    #df_fig['score'] = df_fig.groupby('work_category')['score'].transform(lambda x: (x - x.min()) / (x.max() - x.min()) * 9 + 1)
    
    fig = px.line(df_fig, x = 'decade', y = 'score', color = 'work_category',
                  width = 1200, 
                  height = 600, 
                  title = region_name, 
                  template = 'simple_white')
                  #line_shape='spline')
    
    fig.update_layout(legend=dict(title=dict(text='Identifiers Continent')))
    fig.update_layout(xaxis=dict(dtick=100))
    fig.update_layout(xaxis_title='', yaxis_title='Normalised Score')
    
    return fig

In [407]:
fig = make_figure(df_trend, region_name = 'Korea', min_decade = 600, max_decade = 1800,  work_category = ['musical work/composition'])

In [471]:
df_count_work = df[['individual_wikidata_id', 'individual_name', 'work_wikidata_id']].drop_duplicates()
df_count_work = df_count_work.groupby(['individual_wikidata_id', 'individual_name'])['work_wikidata_id'].count().rename('count_works').reset_index()

df_individual = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_individual = df_individual[['individual_wikidata_id','individual_name', 'birthyear']]

df_count_work = pd.merge(df_count_work, df_individual, on = ['individual_wikidata_id', 'individual_name'], how = 'outer')
df_count_work = df_count_work.sort_values('count_works', ascending=False).reset_index(drop=True)
df_count_work['count_works'] = df_count_work['count_works'].fillna(0)


df_individual_region_group = df_individual_region.groupby('individual_wikidata_id')['region_name'].apply(lambda x : ' | '.join(x))
df_count_work = pd.merge(df_count_work, df_individual_region_group, on = ['individual_wikidata_id'])
df_count_work = df_count_work.drop_duplicates('individual_wikidata_id', keep = 'first')